In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [116]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.sql import label

In [142]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite",echo=True)
this_conn = engine.connect()

2021-07-23 12:27:40,846 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-07-23 12:27:40,849 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:27:40,851 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-07-23 12:27:40,853 INFO sqlalchemy.engine.base.Engine ()


In [143]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

2021-07-23 12:27:47,150 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-07-23 12:27:47,154 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:27:47,162 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("measurement")
2021-07-23 12:27:47,164 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:27:47,171 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-07-23 12:27:47,172 INFO sqlalchemy.engine.base.Engine ('measurement',)
2021-07-23 12:27:47,176 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("measurement")
2021-07-23 12:27:47,177 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:27:47,179 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("measurement")
2021-07-23 12:27:47,180 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:27:47,185 INFO sqlalchemy.engine.base.Engine SELEC

In [144]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [145]:
### Create the inspector and connect it to the engine
inspector = inspect(engine)

In [146]:
# Save references to each table
### Collect the names of tables within the database
inspector.get_table_names()

2021-07-23 12:28:15,190 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-07-23 12:28:15,195 INFO sqlalchemy.engine.base.Engine ()


['measurement', 'station']

In [147]:
# get columns for measurement table
meas_cols = inspector.get_columns('measurement')
for c in meas_cols:
    print(c['name'], c['type'])

2021-07-23 12:28:17,133 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("measurement")
2021-07-23 12:28:17,137 INFO sqlalchemy.engine.base.Engine ()
id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [148]:
# get columns for station table
stat_cols = inspector.get_columns('station')
for c in stat_cols:
    print(c['name'], c['type'])

2021-07-23 12:28:18,896 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("station")
2021-07-23 12:28:18,901 INFO sqlalchemy.engine.base.Engine ()
id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [149]:
# not sure I need these, but doing it because it was in very first activity Query All Records in the Database
measurement_data = engine.execute("SELECT * FROM measurement")
measurement_df = pd.read_sql("SELECT * FROM measurement", this_conn)

measurement_df.head()

2021-07-23 12:28:29,269 INFO sqlalchemy.engine.base.Engine SELECT * FROM measurement
2021-07-23 12:28:29,271 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:28:29,274 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM measurement")
2021-07-23 12:28:29,275 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:28:29,277 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM measurement")
2021-07-23 12:28:29,279 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:28:29,282 INFO sqlalchemy.engine.base.Engine SELECT * FROM measurement
2021-07-23 12:28:29,284 INFO sqlalchemy.engine.base.Engine ()


,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0


In [150]:
# not sure I need these, but doing it because it was in very first activity Query All Records in the Database
station_data = engine.execute("SELECT * FROM station")
station_df = pd.read_sql("SELECT * FROM station", this_conn)

station_df

2021-07-23 12:28:41,782 INFO sqlalchemy.engine.base.Engine SELECT * FROM station
2021-07-23 12:28:41,785 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:28:41,788 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM station")
2021-07-23 12:28:41,790 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:28:41,793 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM station")
2021-07-23 12:28:41,794 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 12:28:41,796 INFO sqlalchemy.engine.base.Engine SELECT * FROM station
2021-07-23 12:28:41,797 INFO sqlalchemy.engine.base.Engine ()


,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,4,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,6,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,7,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,8,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,9,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [151]:
### Assign classes to variables 
Measurement = Base.classes.measurement
Station = Base.classes.station

In [152]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [153]:
### This is the first date as a check to make sure things were working while I searched for last date
# Find the most recent date in the data set.
earliest_date = session.query(Measurement.date).order_by(Measurement.date).first()
print(earliest_date)

2021-07-23 12:29:00,142 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-07-23 12:29:00,145 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date 
FROM measurement ORDER BY measurement.date
 LIMIT ? OFFSET ?
2021-07-23 12:29:00,154 INFO sqlalchemy.engine.base.Engine (1, 0)
('2010-01-01',)


In [154]:
# find most recent date in the database
last_date = session.query(func.max(Measurement.date)).first()
print(last_date)

2021-07-23 12:29:05,351 INFO sqlalchemy.engine.base.Engine SELECT max(measurement.date) AS max_1 
FROM measurement
 LIMIT ? OFFSET ?
2021-07-23 12:29:05,357 INFO sqlalchemy.engine.base.Engine (1, 0)
('2017-08-23',)


## Design a query to retrieve the last 12 months of precipitation data and plot the results. 

In [183]:
# Starting from the most recent data point in the database. 
# The date that was pulled was a tuple & I saw online to use a marker to pull just the date 
# end_date = dt.date(last_date[0])  # The date is 2017-08-23; *the code DOES WORK, but I hard-coded to be sure*
end_hand = dt.date(2017, 8, 23)
# Calculate the date one year from the last date in data set.
# start_date = end_date - dt.timedelta(days=365)  # THIS date should be 2016-08-24; hadn't gotten to work yet
start_hand = dt.date(2016, 8, 24) 
print(f'start date is {start_hand} & end date is {end_hand}')

start date is 2016-08-24 & end date is 2017-08-23


# Starting from the most recent data point in the database. 
# The date that was pulled was a tuple & I saw online to use a marker to pull just the date 
end_date = dt.date(last_date[0])  # The date is 2017-08-23; ***the code DID WORK***
# print(end_date)
# end_hand = dt.date(2017, 8, 23)
# Calculate the date one year from the last date in data set.
start_date = end_date - dt.timedelta(days=365)  # THIS date should be 2016-08-24; hadn't gotten to work yet
# start_hand = dt.date(2016, 8, 24) 
print(f'start date is {start_date} & end date is {end_date}')

sel = [Dow.stock, 
       func.avg(Dow.open_price), 
       func.avg(Dow.high_price), 
       func.avg(Dow.low_price), 
       func.avg(Dow.close_price)]
may_averages = session.query(*sel).\
    filter(func.strftime("%m", Dow.date) == "05").\
    group_by(Dow.stock).\
    order_by(Dow.stock).all()
may_averages

In [201]:
# V0 produces list of tuples; has appropriate dates, but need to collapse so there is only 1 pair per date
precip_output0 = session.query(Measurement.station, Measurement.date, Measurement.prcp).\
    filter(Measurement.date>=start_hand).all()
    # order_by(Measurement.date)
    # all()

2021-07-23 15:11:33,150 INFO sqlalchemy.engine.base.Engine SELECT measurement.station AS measurement_station, measurement.date AS measurement_date, measurement.prcp AS measurement_prcp 
FROM measurement 
WHERE measurement.date >= ?
2021-07-23 15:11:33,154 INFO sqlalchemy.engine.base.Engine ('2016-08-24',)


In [202]:
print(precip_output0)

[('USC00519397', '2016-08-24', 0.08), ('USC00519397', '2016-08-25', 0.08), ('USC00519397', '2016-08-26', 0.0), ('USC00519397', '2016-08-27', 0.0), ('USC00519397', '2016-08-28', 0.01), ('USC00519397', '2016-08-29', 0.0), ('USC00519397', '2016-08-30', 0.0), ('USC00519397', '2016-08-31', 0.13), ('USC00519397', '2016-09-01', 0.0), ('USC00519397', '2016-09-02', 0.0), ('USC00519397', '2016-09-03', 0.0), ('USC00519397', '2016-09-04', 0.03), ('USC00519397', '2016-09-05', None), ('USC00519397', '2016-09-06', None), ('USC00519397', '2016-09-07', 0.05), ('USC00519397', '2016-09-08', 0.0), ('USC00519397', '2016-09-09', 0.03), ('USC00519397', '2016-09-10', 0.0), ('USC00519397', '2016-09-11', 0.05), ('USC00519397', '2016-09-12', 0.0), ('USC00519397', '2016-09-13', 0.02), ('USC00519397', '2016-09-14', 1.32), ('USC00519397', '2016-09-15', 0.42), ('USC00519397', '2016-09-16', 0.06), ('USC00519397', '2016-09-17', 0.05), ('USC00519397', '2016-09-18', 0.0), ('USC00519397', '2016-09-19', 0.0), ('USC0051939

In [171]:
# V1 * * * This version creates empty list
# INSTRUCTIONS: Perform a query to retrieve the data and precipitation scores
# So what you are actually asking for is a sum across all of the stations? 
# As opposed to plotting each station separately?

## currently has HARD-CODED date

precip_output1 = session.query(Measurement.date, func.sum(Measurement.prcp)).\
    filter(func.strftime("%m", Measurement.date) >= start_hand).\
    group_by(Measurement.date).\
    order_by(Measurement.date).all() # when I take out .all(), it doesn't seem to do anything

2021-07-23 13:42:11,228 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE strftime(?, measurement.date) >= ? GROUP BY measurement.date ORDER BY measurement.date
2021-07-23 13:42:11,230 INFO sqlalchemy.engine.base.Engine ('%m', '2016-08-24')


In [ ]:
# V2 * * * This version creates a list of tuples, BUT has ALL DATES; 
# difference btwn V1 & V2 is where parens are in filter line

## currently has HARD-CODED date
#precip_output = session.query(Measurement.date, Measurement.station, Measurement.prcp).\

precip_output2 = session.query(Measurement.date, func.sum(Measurement.prcp)).\
    filter(func.strftime("%m", Measurement.date >= start_hand)).\
    group_by(Measurement.date).\
    order_by(Measurement.date).all()
    # group_by(Measurement.date) # returns only the first station's measurements -- WHY?!?!

2021-07-23 13:42:33,389 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE strftime(?, measurement.date >= ?) GROUP BY measurement.date ORDER BY measurement.date
2021-07-23 13:42:33,392 INFO sqlalchemy.engine.base.Engine ('%m', '2016-08-24')


In [175]:
# V3 * * * This version doesn't even seem to populate the filter

## currently has HARD-CODED date
#precip_output = session.query(Measurement.date, Measurement.station, Measurement.prcp).\

precip_output3 = session.query(Measurement.date, func.sum(Measurement.prcp)).\
    filter(func.strftime("%m", Measurement.date >= start_hand)).\
    order_by(Measurement.date)#.all()


In [178]:
# V4 * * * This version doesn't even seem to populate the filter

## currently has HARD-CODED date
#precip_output = session.query(Measurement.date, Measurement.station, Measurement.prcp).\

precip_output4 = session.query(Measurement.date, func.sum(Measurement.prcp)).\
    filter(func.strftime("%m", Measurement.date >= start_hand)) #.\
    # order_by(Measurement.date)#.all()


In [172]:
# It's not running the filter when I put .all(), but when I don't put .all(), it won't show me anything
print(precip_output1)

[]


In [174]:
print(precip_output2)

[('2010-01-01', 1.06), ('2010-01-02', 0.02), ('2010-01-03', 0.0), ('2010-01-04', 0.01), ('2010-01-05', 0.03), ('2010-01-06', 0.5800000000000001), ('2010-01-07', 1.65), ('2010-01-08', 0.06), ('2010-01-09', 0.02), ('2010-01-10', 0.02), ('2010-01-11', 0.8800000000000001), ('2010-01-12', 0.02), ('2010-01-13', 0.02), ('2010-01-14', 0.0), ('2010-01-15', 0.0), ('2010-01-16', 0.0), ('2010-01-17', 0.0), ('2010-01-18', 0.0), ('2010-01-19', 0.0), ('2010-01-20', 0.01), ('2010-01-21', 0.65), ('2010-01-22', 0.0), ('2010-01-23', 0.0), ('2010-01-24', 0.02), ('2010-01-25', 0.0), ('2010-01-26', 0.25), ('2010-01-27', 1.6), ('2010-01-28', 0.73), ('2010-01-29', 2.19), ('2010-01-30', 7.3500000000000005), ('2010-01-31', 2.16), ('2010-02-01', 0.05), ('2010-02-02', 0.05), ('2010-02-03', 3.21), ('2010-02-04', 0.02), ('2010-02-05', 0.16), ('2010-02-06', 0.12), ('2010-02-07', 0.01), ('2010-02-08', 0.0), ('2010-02-09', 0.0), ('2010-02-10', 0.0), ('2010-02-11', 0.0), ('2010-02-12', 0.8200000000000001), ('2010-02-13

In [176]:
print(precip_output3)

SELECT measurement.date AS measurement_date, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE strftime(?, measurement.date >= ?) ORDER BY measurement.date


In [179]:
print(precip_output4)

SELECT measurement.date AS measurement_date, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE strftime(?, measurement.date >= ?)


In [205]:
# Save the query results as a Pandas DataFrame and set the index to the date column 
# -- which means precipitation is summed on date above
# **trying to pass list of tuples** 7-22; 7-23 now an empty list
precipitation_df = pd.DataFrame(precip_output0)
precipitation_df

,station,date,prcp
0,USC00519397,2016-08-24,0.08
1,USC00519397,2016-08-25,0.08
2,USC00519397,2016-08-26,0.00
3,USC00519397,2016-08-27,0.00
4,USC00519397,2016-08-28,0.01
...,...,...,...
2218,USC00516128,2017-08-19,0.09
2219,USC00516128,2017-08-20,NaN
2220,USC00516128,2017-08-21,0.56
2221,USC00516128,2017-08-22,0.50


In [206]:
# Sort the dataframe by date
precipitation_df = precipitation_df.sort_values(precipitation_df['date'])
# precipitation_df

KeyError: 0       2016-08-24
1       2016-08-25
2       2016-08-26
3       2016-08-27
4       2016-08-28
           ...    
2218    2017-08-19
2219    2017-08-20
2220    2017-08-21
2221    2017-08-22
2222    2017-08-23
Name: date, Length: 2223, dtype: object

In [ ]:
## Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
## Use Pandas to calcualte the summary statistics for the precipitation data
precip_summary_df = precipitation_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()